In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd


# Url from where the table will be scrapped
data_url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
data = requests.get(data_url).text

# define soup object of the url
soup = BeautifulSoup(data, 'xml')
table_contents=[]

# find table in the soup object
table=soup.find('table')

# Search all the postcode, borough, neighborhood 
for row in table.findAll('td'):
    cell = {}
    if row.span.text=='Not assigned':
        pass
    else:
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)

# create a dataframe
df_table=pd.DataFrame(table_contents)


df_table['Borough']=df_table['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})

In [2]:
""" Part 2 Solution for getting longitude and latitude Starts here """

# read csv file to get geolocation details
df_geo=pd.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/labs_v1/Geospatial_Coordinates.csv")

# rename postal code column to match the same column name in above dataframe
df_geo.rename(columns={'Postal Code':'PostalCode'},inplace=True)
df_geo

# merge df_table and geo_df with respect to PostalCode
geo_table_df = pd.merge(df_geo, df_table, on='PostalCode')
geo_table_df
geo_data=geo_table_df[['PostalCode','Borough','Neighborhood','Latitude','Longitude']]
geo_data

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
...,...,...,...,...,...
98,M9N,York,Weston,43.706876,-79.518188
99,M9P,Etobicoke,Westmount,43.696319,-79.532242
100,M9R,Etobicoke,"Kingsview Village, St. Phillips, Martin Grove ...",43.688905,-79.554724
101,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest...",43.739416,-79.588437
